In [67]:
import torch.nn as nn
import librosa
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import IPython.display as ipd

In [68]:
s, sr=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [69]:
X.shape

(513, 2459)

In [70]:
s_abs = np.abs(S)
x_abs = np.abs(X)
s = torch.tensor(np.transpose(s_abs)).cuda()
x = torch.tensor(np.transpose(x_abs)).cuda()

In [71]:
print(s.shape)
print(x.shape)

torch.Size([2459, 513])
torch.Size([2459, 513])


In [72]:
print(x.shape)
# x_dirty = x[:,np.newaxis,:]
# print(x_dirty.shape)

torch.Size([2459, 513])


In [73]:
BATCH = 128
train_loader = torch.utils.data.DataLoader(x, batch_size=BATCH)
test_loader = torch.utils.data.DataLoader(s, batch_size=BATCH)

In [74]:
class NeuralNetwork(nn.Module):
  
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=513, out_channels=5, kernel_size = 3)
#     self.pool1 = nn.MaxPool1d(kernel_size = 2, stride=2)
#     self.conv2 = nn.Conv1d(in_channels=5, out_channels=32, kernel_size = 8)
#     self.pool2 = nn.MaxPool1d(kernel_size = 2, stride=2)
    
#     self.fc1 = nn.Linear(in_features=1000, out_features=513)
    
    self.activation = nn.ReLU()
  
  def forward(self, data):
#     print(data.shape)
    data = data.unsqueeze(0)
    data = data.view(-1, 513).cuda()
    print(data.shape)
    output = self.activation(self.conv1(data))
    print(output.shape)
#     output = self.activation(self.pool1(output))
#     output = self.activation(self.conv2(output))
#     output = self.activation(self.pool2(output))
#     output = self.activation(self.fc1(output))
    final = torch.nn.functional.relu(output)
    return final

In [75]:
# using Adam with learning rat .001 keeping everything else default.
neural_network = NeuralNetwork()
neural_network = neural_network.cuda()
loss_function= nn.MSELoss()
para = neural_network.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.001)
loss_list = []
epochs = 1
for i in range(epochs):
    input_iter = iter(train_loader)
    target_iter = iter(test_loader)
    while True:
        try:
            input_set = input_iter.next()
            target_set = target_iter.next()
            input_set = input_set.cuda()
            target_set = target_set.cuda()
            optimizer.zero_grad()
            network_output  = neural_network(input_set)
            loss = loss_function(network_output , target_set)
            loss.backward()
            optimizer.step()
        except StopIteration:
            break
    loss_list.append(loss.data.cpu().numpy())
print(loss.data)
plt.plot(range(epochs), loss_list)
plt.show()   

torch.Size([128, 513])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [5, 513, 3], but got 2-dimensional input of size [128, 513] instead